In [9]:
# Dataset Path
DatasetPath = "E:/An toàn thông tin/Code/Dataset/NSL-KDD Processed/Final - For Using/"
train_dataset_path = DatasetPath + "Trainset/" + "IDS.csv"
test_dataset_path = DatasetPath + "Testset/" + "KDDTest+.csv"

# Save Model Path
SavedModelPath = "E:/An toàn thông tin/Code/Saved Model/IDSModel/"

In [12]:
import keras
import numpy as np
import pandas as pd
from datetime import date
import os
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Conv1D, Dropout
#from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix




ModuleNotFoundError: No module named 'keras.layers.normalization'

In [8]:
pip install BatchNormalization

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement BatchNormalization (from versions: none)
ERROR: No matching distribution found for BatchNormalization


In [13]:
import keras

print("Phiên bản Keras:", keras.__version__)


Phiên bản Keras: 2.15.0


In [11]:
from keras.layers.normalization import BatchNormalization

ModuleNotFoundError: No module named 'keras.layers.normalization'

In [6]:
def load_dataset_only(path, attack_category):
    if attack_category != 'DOS' and attack_category != 'U2R_AND_R2L':
      raise ValueError("Preprocess Data Fail: Invalid Attack Category")
    df = pd.read_csv(path)
    df = df[df['class'].isin(['Normal', attack_category])]
    print(f" - Amount of Normal record: {len(df[df['class'] == 'Normal'])}")
    print(f" - Amount of {attack_category} record: {len(df[df['class'] == attack_category])}")
    X = np.asarray(df[df.columns[df.columns != "class"]])
    Y = np.asarray(df['class'].map(lambda x : 0 if x == "Normal" else 1))
    return X, Y
def load_dataset(path, attack_category):
    X, Y = load_dataset_only(path, attack_category)
    num_classes = 2
    img_rows, img_cols = 1, X.shape[1]
    X = X.reshape(X.shape[0], img_cols, img_rows)
    Y = keras.utils.to_categorical(np.asarray(Y), num_classes)
    return X, Y

#**1. Define Model**

In [7]:
img_rows, img_cols = 1, 41
num_classes = 2

# Create Model
model = Sequential()
model.add(Conv1D(128, (4), input_shape=(img_cols, img_rows), activation='tanh'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics= ['accuracy']
            )
model.summary()

NameError: name 'Sequential' is not defined

#**Train**

In [ ]:
# Train
today = str(date.today())
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
batch_size = 128
epochs = 10

for attack_category in ['DOS', 'U2R_AND_R2L']:
    print(f"{100*'-'}\nModel: CNN, Attack Category: {attack_category}")
    # Load data
    print("Trainset")
    x_train, y_train = load_dataset(trainset_path,attack_category)
    print("Testset")
    x_test, y_test = load_dataset(testset_path,attack_category)
    # Train Model
    model.fit(x_train,
            y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_split=0.25,
            callbacks=[es])

    result = model.evaluate(x_test, y_test, batch_size=batch_size)
    print(f"Loss: {result[0]:.2f} - Acc: {result[1]*100:.2f}%")
    model_saved_name = str(f"created_date_{today}_CNN.h5")
    model_saved_folder = str(f"{save_model_path}{attack_category}/Deep_Learning/Keras/")
    if not os.path.exists(model_saved_folder):
                os.makedirs(model_saved_folder)
    model.save(f"{model_saved_folder}{model_saved_name}")
    print(f"Saved model to disk\n Folder: {model_saved_folder}\n Name: {model_saved_name}")

----------------------------------------------------------------------------------------------------
Model: CNN, Attack Category: DOS
Trainset
 - Amount of Normal record: 33671
 - Amount of DOS record: 22963
Testset
 - Amount of Normal record: 9711
 - Amount of DOS record: 7460
Train on 42475 samples, validate on 14159 samples
Epoch 1/10
42475/42475 [==============================] - 11s 268us/step - loss: 0.0107 - accuracy: 0.9962 - val_loss: 0.0204 - val_accuracy: 0.9948
Epoch 2/10
42475/42475 [==============================] - 11s 266us/step - loss: 0.0093 - accuracy: 0.9970 - val_loss: 0.0222 - val_accuracy: 0.9911
Epoch 00002: early stopping
17171/17171 [==============================] - 1s 54us/step
Loss: 0.47 - Acc: 91.37%
Saved model to disk
 Folder: /content/drive/My Drive/Study/KLTN/Saved Model/IDSModel/DOSDeep_Learning/Keras/
 Name: created_date_2020-07-17_CNN.h5
----------------------------------------------------------------------------------------------------
Model: CNN, 

Thư mục lưu sai, đã chỉnh lại code và move model về đúng thư mục.  
"/content/drive/My Drive/Study/KLTN/Saved Model/IDSModel/***DOSDeep_Learning***/Keras/"  
--> /content/drive/My Drive/Study/KLTN/Saved Model/IDSModel/***DOS/Deep_Learning***/Keras/

#**Calc DR**

In [ ]:
def create_batch(x, y, batch_size):
    a = list(range(len(x)))
    np.random.shuffle(a)
    x = x[a]
    y = y[a]

    batch_x = [x[batch_size * i : (i+1)*batch_size,:].tolist() for i in range(len(x)//batch_size)]
    batch_y = [y[batch_size * i : (i+1)*batch_size].tolist() for i in range(len(x)//batch_size)]
    return np.array(batch_x), np.array(batch_y)

In [ ]:
# Calc DR
print(f"{100*'='}\nCALCULATE DETECTION RATE")
for attack_category in ['DOS', 'U2R_AND_R2L']:
    print(f"{100*'-'}\nModel: CNN, Attack Category: {attack_category}")
    
    # Load Data
    print("Testset")
    x_test, y_test = load_dataset_only(testset_path,attack_category)
    batch_x_test, batch_y_test = create_batch(x_test, y_test, batch_size = 128)
    

    # Load Model
    model_saved_name = str(f"created_date_{today}_CNN.h5")
    ids_model_path = str(f"{save_model_path}{attack_category}Deep_Learning/Keras/{model_saved_name}")
    ids_model = load_model(ids_model_path)
    # ids_model.summary()

    # Test & Calc DR
    drs = []
    for idx, data in enumerate(batch_x_test):
        true_labels = batch_y_test[idx]
        data = data.reshape(data.shape[0], 41, 1)
        ids_outputs = ids_model.predict(data)
        predict_labels = np.argmax((ids_outputs),axis = 1)

        tn, fn, fp, tp = confusion_matrix(true_labels,predict_labels).ravel()

        drs.append(tp/(tp + fp))
    dr = np.mean(drs)*100
    print(f" -> {attack_category} - DR: {dr:.2f}%")

CALCULATE DETECTION RATE
----------------------------------------------------------------------------------------------------
Model: CNN, Attack Category: DOS
Testset
 - Amount of Normal record: 9711
 - Amount of DOS record: 7460
 -> DOS - DR: 81.73%
----------------------------------------------------------------------------------------------------
Model: CNN, Attack Category: U2R_AND_R2L
Testset
 - Amount of Normal record: 9711
 - Amount of U2R_AND_R2L record: 2952
 -> U2R_AND_R2L - DR: 0.13%
